In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW inhert_ig_tags_from_mill AS
SELECT *
FROM VALUES
  ('AccessionNbr', '4_prod', 'raw', 'mill_clinical_event', 'REFERENCE_NBR'),
  ('Clinical_Event_ID', '4_prod', 'raw', 'mill_clinical_event', 'CLINICAL_EVENT_ID'),
  ('EVENT_ID', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_ID'),
  ('ENCNTR_ID', '4_prod', 'raw', 'mill_clinical_event', 'ENCNTR_ID'),
  ('PARENT_EVENT_ID', '4_prod', 'raw', 'mill_clinical_event', 'PARENT_EVENT_ID'),
  ('ExamCode', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_CD'),
  ('ExamCodeSeq', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_CD'),
  ('MillRefNbr', '4_prod', 'raw', 'mill_clinical_event', 'REFERENCE_NBR'),
  ('MillPersonId', '4_prod', 'raw', 'mill_clinical_event', 'PERSON_ID'),
  ('PacsPatientId', '4_prod', 'raw', 'mill_clinical_event', 'PERSON_ID'),
  ('MillEventDate', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_START_DT_TM'),
  ('RequestId', '4_prod', 'raw', 'mill_clinical_event', 'CLINICAL_EVENT_ID'),
  ('RequestQuestion', '4_prod', 'raw', 'mill_ce_blob', 'BLOB_CONTENTS'),
  ('RequestAnamnesis', '4_prod', 'raw', 'mill_ce_blob', 'BLOB_CONTENTS'),
  ('ExaminationIdString', '4_prod', 'raw', 'mill_clinical_event', 'CLINICAL_EVENT_ID'),
  ('ExaminationStudyUid', '4_prod', 'raw', 'mill_clinical_event', 'CLINICAL_EVENT_ID'),
  ('ExaminationDescription', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_TITLE_TEXT'),
  ('ExaminationModality', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_CD'),
  ('ExaminationBodyPart', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_CD'),
  ('PacsReportCount', '4_prod', 'raw', 'mill_clinical_event', 'EVENT_CD')
AS t(pacs_col, mill_catalog, mill_schema, mill_table, mill_col);

In [0]:
# Load the temporary view as a DataFrame
inhert_ig_tags_from_mill = spark.table("inhert_ig_tags_from_mill")

for row in inhert_ig_tags_from_mill.collect():
    pacs_col = row['pacs_col']
    mill_catalog = row['mill_catalog']
    mill_schema = row['mill_schema']
    mill_table = row['mill_table']
    mill_col = row['mill_col']

    # Query the column_tags table for tags
    tags_df = spark.sql(
        f"""
        SELECT tag_name, tag_value
        FROM system.information_schema.column_tags
        WHERE catalog_name = '{mill_catalog}'
          AND schema_name = '{mill_schema}'
          AND table_name = '{mill_table}'
          AND column_name = '{mill_col}'
        """
    )

    mill_col_tags = {
        row['tag_name']: row['tag_value']
        for row in tags_df.collect()
    }

    # Prepare tag assignments if tags exist
    if 'ig_risk' in mill_col_tags:
        set_tags.append(f"")
        set_tags_sql = ', '.join(set_tags)
        q = f"""
            ALTER TABLE 4_prod.pacs.hdruk_barts_imaging_metadata
            ALTER COLUMN `{pacs_col}`
            SET TAGS ('ig_risk' = '{mill_col_tags['ig_risk']}')
            """
        spark.sql(q)

    if 'ig_severity' in mill_col_tags:
        set_tags.append(f"")
        set_tags_sql = ', '.join(set_tags)
        q = f"""
            ALTER TABLE 4_prod.pacs.hdruk_barts_imaging_metadata
            ALTER COLUMN `{pacs_col}`
            SET TAGS ('ig_severity' = '{mill_col_tags['ig_severity']}')
            """
        spark.sql(q)
